In [12]:
### info from oxyphen developer:

#To run multinome version, you just have to run the script oxyphen_mutlinome.py 
#and in SETTINGS file add a line (“PROTEOMES_FOLDER”) that points to the folder with your proteomes in fasta format.
#SETTINGS:
#INPUT_FILE=/Users/jagodajablonska/Desktop/oxygen_paper/oxyphen/PROTEOMES/UP000245539_1247513.fasta
#PROTEOMES_FOLDER=/Users/jagodajablonska/Desktop/oxygen_paper/oxyphen/PROTEOMES
#BLAST_PATH=/usr/local/ncbi/blast/bin
#NUM_THREADS=2

#The program will save separate files with output oxygen-utilizing EC classes in OUTPUT directory 
#(and detailed mapping) for every input proteome and display the number of them on the screen. 

#I want to avoid returning explicitly whether the organism is aerobic or not.  
#I’d stick to the publication data and assume that an organism with more than around 25 oxygen-utillizing enzymes 
#is with high confidence an aerobe.

#and here's the BLAST executables handbook. Python script below is running command line blast commands from downloaded blast exe
# http://nebc.nerc.ac.uk/bioinformatics/documentation/blast+/user_manual.pdf

In [1]:
# notes to self:
#before running this script, complete four steps:

#Use API scrape (separate jupyter notebook) to get proteome csv from uniprot website
#use for loop script (R) to convert csv files to fasta files.
#rename / discard old folders and files, and remake new OUTPUT folder.
#edit line 122-123 below to set protein homology threshold for greater specificity (high, 60) or sensitivity (low, 40).

#after running this oxyphen multiome script, run another R script to merge results summary table with org phylogeny files


In [7]:
import os
path='/Users/tonglen/downloads/oxyphen-multiome/'
os.chdir(path)

os.getcwd()

'/Users/tonglen/Downloads/oxyphen-multiome'

In [8]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_selection import RFE
from Bio.ExPASy import Enzyme
import numpy as np
import pandas as pd
import os, glob

In [9]:
CONFIG_FILE = open("SETTINGS", "r").read().splitlines()

if os.path.isfile("OUTPUT/results_table.tsv"):
    print ("OUTPUT FILE EXISTS, PLEASE MOVE/REMOVE YOUR PREVIOUS RESULTS!!!")
    exit()
else:
    GLOBAL_RESULTS = open("OUTPUT/results_table.tsv", "a")

def read_config():
    multinome_folder=""
    for line in CONFIG_FILE:
        if line.startswith("BLAST_PATH"):
            blast_path = line.split("=")[1]
        if line.startswith("NUM_THREADS"):
            num_threads = float(line.split("=")[1])
        if line.startswith("PROTEOMES_FOLDER") and line.split("=")[1]:
            ### enter multinome mode
            multinome_folder = line.split("=")[1]

    return blast_path, num_threads, multinome_folder

def do_oxyphen(proteome, output_filename, ec_classes_file):

    '''
    Read and parse enzyme.dat file
    '''
    input_name = "DATA/enzyme.dat"
    output_name = 'DATA/ec_uniprot.tsv'

    ### program ###
    handle = open(input_name)
    records = Enzyme.parse(handle)

    out = dict() #dict of dicts, first key: EC number, second key: field
    transferred = dict() #dict of lists
    for record in records:
        if 'Transferred entry:' in record['DE']:
            record['DE'] = record['DE'].rstrip('.') #remove period
            record['DE'] = record['DE'].replace('Transferred entry:',' ') #remove title
            record['DE'] = record['DE'].replace(',',' ') #remove commas
            record['DE'] = record['DE'].replace('and',' ') #remove and
            point_to = record['DE'].split()
            transferred[record['ID']] = point_to
        else:
            out[record['ID']] = dict()
            out[record['ID']]['uniprot'] = ' '.join([x[0] for x in record['DR']])
            out[record['ID']]['description'] = record['DE']
            out[record['ID']]['transferred'] = False

    for id in transferred:
        out[id] = dict()
        out[id]['uniprot'] = ' '.join([out[x]['uniprot'] for x in transferred[id]])
        out[id]['description'] = 'Transferred entry: ' + ' '.join(transferred[id])
        out[id]['transferred'] = True
    df = pd.DataFrame.from_dict(out, orient='index')
    df.index.name = 'EC'
    df.to_csv(output_name, sep='\t')
    

    '''
    Take a subset of ecs of interest
    '''

    oxidases = tuple(open("DATA/oxygen_ecclasses", "r").read().splitlines())

    infile = open("DATA/ec_uniprot.tsv", "r").readlines()
    outfile = open("DATA/ec_uniprot_oxidases.tsv", "w")

    for line in infile:
        if line.startswith("EC"):
            outfile.write(line)
        elif line.startswith(oxidases):
            outfile.write(line)

    outfile.close()
    

    '''
    write a file with one uniprot ID per line, containing all of the
    uniprot IDs mentioned in uniprot column of the input file

    Ignore EC numbers that have been transferred
    '''

    input = "DATA/ec_uniprot_oxidases.tsv"
    output = "DATA/uniprot_ids.txt"

    df = pd.read_table(input)
    df.dropna(subset=['uniprot'], inplace=True) #ignore EC numbers with no uniprot ids associated

    df = df[df.transferred == False] #ignore EC numbers that are obsolete due to transfer

    unique_uniprot = set(" ".join(df.uniprot.values).split(" "))

    with open(output, "w") as outfile:
        for id in unique_uniprot:
            outfile.write(id + "\n")
    outfile.close()
    

    '''
    Make blastdb out of the swissprot subset
    '''

    blast_path, num_threads, multinome_folder = read_config()

    os.system("%s -in DATA/sprot_subset.fasta -dbtype prot -out DATA/sprot_subset -hash_index" % (os.path.join(blast_path, "makeblastdb")))

    '''
    Blast our pre-selected proteomes against the uniprot subset
    '''
    #proteome = "/Users/tonglen/Downloads/oxyphen-multiome/PROTEOMES/1131.fasta"
    print ("Performing Blast searches against oxygen-utilizing database...")
    os.system("%s -max_target_seqs 1 -outfmt '6 qseqid sseqid pident evalue qcovs' -query %s -db DATA/sprot_subset -out DATA/new_sequences_sprot_enzyme.tab -num_threads %d" % (os.path.join(blast_path, "blastp"), proteome, num_threads) )
   #os.system("%s -max_target_seqs 1 -outfmt '6 qseqid sseqid pident evalue qcovs' -query %s -db DATA/sprot_subset -out DATA/new_sequences_sprot_enzyme.tab -num_threads %d" % (os.path.join(blast_path, "blastp"), proteome, num_threads) )


    '''
    Filter Blast output.
    '''
    evalue = 0.001
    identity = 60.0
    coverage = 60.0

    print ("Filtering Blast output: evalue",evalue, " identity", identity, " coverage", coverage)
    hits_table_file_name = "DATA/new_sequences_sprot_enzyme.tab"
    hits_table_file_name_filtered_out = open("DATA/new_sequences_sprot_enzyme_filtered.tab", "w")

    hits_table_file_name_filtered_out.write("\t".join(["hit","subject","id","len","eval","cov"])+"\n")

    for line in open(hits_table_file_name, "r").read().splitlines():
        if line.startswith("#"): continue

        query, target, ident, eval, cover = line.split("\t")
        eval = float(eval)
        ident = float(ident)
        cover = float(cover)

        if eval <= evalue and ident >= identity and cover >= coverage:
            hits_table_file_name_filtered_out.write(line+"\n")

    hits_table_file_name_filtered_out.close()
    
    

    hits_table_file_name_filtered = "DATA/new_sequences_sprot_enzyme_filtered.tab"
    enzyme_table_file_name = 'DATA/ec_uniprot_oxidases.tsv'

    hits = pd.read_csv(hits_table_file_name_filtered, sep="\t", header=0)
    enzyme = pd.read_csv(enzyme_table_file_name, sep="\t", header=0)

    hits.fillna('', inplace=True)  #replace empty values with blank spaces
    enzyme.fillna('', inplace=True)

    enzyme = enzyme[enzyme.transferred == False] #drop transferred EC numbers

    hits.subject = hits.subject.str[3:9] #take just the uniprot ID from the name

    def get_ecs(uniprot):
        if uniprot == '': #ignore invalid uniprot ids
            return ''
        else:
            return ' '.join(enzyme.EC[enzyme.uniprot.str.contains(uniprot)].values)

    hits['EC'] = hits.subject.apply(get_ecs)

    output_file_name = output_filename
    hits.to_csv(output_file_name, sep="\t", index=False)
    


    ### read final mapping output

    mapping_out = open(output_file_name, "r").read().splitlines()
    ecs_dict = {}

    for line in mapping_out[1:]:
        splitted = line.split("\t")
        ecs = splitted[-1]

        for ec in ecs.split():
            if ec not in ecs_dict:
                ecs_dict[ec] = []
            ecs_dict[ec].append(splitted[0])

    print ("\n\n")
    print (len(ecs_dict), "oxygen-utilizing enzymes were found from classes", ecs_dict.keys())
    
    ec_out = open(ec_classes_file, "w")  
    ec_out.write("\t".join(ecs_dict.keys()))

    ec_out.close()
    
    GLOBAL_RESULTS.write(os.path.basename(proteome)+"\t"+str(len(ecs_dict))+"\t"+",".join(ecs_dict.keys())+"\n")
    
    print ("Detailed mapping can be found in OUTPUT/oxygen_utilizing_annot.tsv file")
    print ("Executing SVM classifier...")

    infile = open("DATA/model_svm", "r").read().splitlines()

    classifier_input = []
    classes = []
    ec_classes = []

    for line in infile:

        if line.startswith("@attribute") and "class" not in line:
            ec_classes.append(line.split()[1].replace("'",""))

'''
Execute oxyphen for all proteomes in the input folder
'''            

def do_for_all():
    blast_path, num_threads, multinome_folder, = read_config()  
    proteomes = glob.glob(os.path.join(multinome_folder, "*"))
    #proteomes = glob.glob(os.path.join("Users/tonglen/Downloads/oxyphen-multiome/TEST", "*.fasta")
    #print ("PROTEOMES IN YOUR PROTEOMES_FOLDER DIRECTORY:")

    for proteome in proteomes:
        fname = os.path.splitext(os.path.basename(proteome))[0]
        output_filename = os.path.join("OUTPUT",fname+"_oxygen_utilizing_annot.tsv")

        ec_classes_file = os.path.join("OUTPUT",fname+"_EC_CLASSES.txt")

        print ("\n\n RUNNING OXYPHEN FOR PROTEOME %s" % proteome)
        do_oxyphen(proteome, output_filename, ec_classes_file)

        print ("\n\n OUTPUT MAPPING FILE FOR THIS PROTEOME CAN BE FOUND IN %s" % output_filename)
        print ("LIST OF EC CLASSES FOR THIS PROTEOME CAN BE FOUND IN %s" % ec_classes_file)
        
    GLOBAL_RESULTS.close()

if __name__ == '__main__':
    do_for_all()

print ("YOUR OUTPUT FILES CAN BE FOUND IN OUTPUT FOLDER")



 RUNNING OXYPHEN FOR PROTEOME /Users/tonglen/Downloads/oxyphen-multiome/PROTEOMES/1076588.fasta
Performing Blast searches against oxygen-utilizing database...
Filtering Blast output: evalue 0.001  identity 60.0  coverage 60.0



8 oxygen-utilizing enzymes were found from classes dict_keys(['1.14.13.7', '1.15.1.1', '4.1.1.39', '1.4.3.16', '1.14.13.236', '1.9.3.1', '1.3.3.3', '1.10.2.2'])
Detailed mapping can be found in OUTPUT/oxygen_utilizing_annot.tsv file
Executing SVM classifier...


 OUTPUT MAPPING FILE FOR THIS PROTEOME CAN BE FOUND IN OUTPUT/1076588_oxygen_utilizing_annot.tsv
LIST OF EC CLASSES FOR THIS PROTEOME CAN BE FOUND IN OUTPUT/1076588_EC_CLASSES.txt


 RUNNING OXYPHEN FOR PROTEOME /Users/tonglen/Downloads/oxyphen-multiome/PROTEOMES/1163617.fasta
Performing Blast searches against oxygen-utilizing database...
Filtering Blast output: evalue 0.001  identity 60.0  coverage 60.0



9 oxygen-utilizing enzymes were found from classes dict_keys(['1.11.1.21', '1.4.3.16', '4.1.1.3

Filtering Blast output: evalue 0.001  identity 60.0  coverage 60.0



48 oxygen-utilizing enzymes were found from classes dict_keys(['1.9.3.1', '1.5.3.1', '1.13.11.11', '1.14.99.46', '1.15.1.1', '1.16.3.1', '1.10.3.10', '1.10.2.2', '1.3.3.11', '1.14.13.149', '1.13.11.5', '1.14.12.10', '1.13.11.3', '1.3.99.28', '1.10.3.4', '1.14.13.208', '1.14.13.29', '1.14.13.227', '1.11.1.21', '1.14.14.11', '1.13.11.8', '1.14.12.13', '1.4.3.5', '1.13.11.6', '1.3.3.3', '1.10.3.14', '1.13.12.16', '1.14.13.81', '1.14.12.7', '1.7.2.1', '1.14.13.2', '4.1.1.39', '1.10.3.9', '1.14.11.1', '1.14.14.9', '1.11.1.6', '1.4.3.16', '1.14.13.23', '1.1.3.47', '1.2.3.1', '1.1.3.17', '1.4.3.2', '1.5.3.7', '1.1.3.15', '1.10.3.11', '1.13.11.27', '1.1.5.4', '1.13.11.79'])
Detailed mapping can be found in OUTPUT/oxygen_utilizing_annot.tsv file
Executing SVM classifier...


 OUTPUT MAPPING FILE FOR THIS PROTEOME CAN BE FOUND IN OUTPUT/1904441_oxygen_utilizing_annot.tsv
LIST OF EC CLASSES FOR THIS PROTEOME CAN BE FOUND IN OUT

Performing Blast searches against oxygen-utilizing database...
Filtering Blast output: evalue 0.001  identity 60.0  coverage 60.0



13 oxygen-utilizing enzymes were found from classes dict_keys(['1.11.1.6', '1.9.3.1', '1.13.11.79', '1.10.2.2', '1.16.3.1', '1.5.3.1', '1.15.1.1', '4.1.1.39', '1.3.99.28', '1.14.13.227', '1.14.14.9', '1.4.3.5', '1.3.3.11'])
Detailed mapping can be found in OUTPUT/oxygen_utilizing_annot.tsv file
Executing SVM classifier...


 OUTPUT MAPPING FILE FOR THIS PROTEOME CAN BE FOUND IN OUTPUT/349101_oxygen_utilizing_annot.tsv
LIST OF EC CLASSES FOR THIS PROTEOME CAN BE FOUND IN OUTPUT/349101_EC_CLASSES.txt


 RUNNING OXYPHEN FOR PROTEOME /Users/tonglen/Downloads/oxyphen-multiome/PROTEOMES/349102.fasta
Performing Blast searches against oxygen-utilizing database...
Filtering Blast output: evalue 0.001  identity 60.0  coverage 60.0



13 oxygen-utilizing enzymes were found from classes dict_keys(['1.7.2.1', '1.11.1.21', '4.1.1.39', '1.9.3.1', '1.16.3.1', '1.10.2.2',

Performing Blast searches against oxygen-utilizing database...
Filtering Blast output: evalue 0.001  identity 60.0  coverage 60.0



8 oxygen-utilizing enzymes were found from classes dict_keys(['4.1.1.39', '1.4.3.16', '1.7.2.1', '1.3.3.3', '1.1.5.4', '1.10.3.14', '1.15.1.1', '1.9.3.1'])
Detailed mapping can be found in OUTPUT/oxygen_utilizing_annot.tsv file
Executing SVM classifier...


 OUTPUT MAPPING FILE FOR THIS PROTEOME CAN BE FOUND IN OUTPUT/580332_oxygen_utilizing_annot.tsv
LIST OF EC CLASSES FOR THIS PROTEOME CAN BE FOUND IN OUTPUT/580332_EC_CLASSES.txt


 RUNNING OXYPHEN FOR PROTEOME /Users/tonglen/Downloads/oxyphen-multiome/PROTEOMES/637389.fasta
Performing Blast searches against oxygen-utilizing database...
Filtering Blast output: evalue 0.001  identity 60.0  coverage 60.0



1 oxygen-utilizing enzymes were found from classes dict_keys(['4.1.1.39'])
Detailed mapping can be found in OUTPUT/oxygen_utilizing_annot.tsv file
Executing SVM classifier...


 OUTPUT MAPPING FILE FOR